### 数据加载

In [1]:
from source.load_excel_to_table import load_excel_to_table

part_a_conn = load_excel_to_table("/Users/sqjian/Documents/codespace/financial_tools/testdata/test.xlsx", "甲方", "part_a")
part_b_conn = load_excel_to_table("/Users/sqjian/Documents/codespace/financial_tools/testdata/test.xlsx", "乙方", "part_b")

### 提取分类条件

In [2]:
import rich
from source.get_group_conditions import get_group_conditions


part_a_group_col_rst = get_group_conditions(part_a_conn, "part_a", "统一仓库", "统一名称", "统一日期")
part_b_group_col_rst = get_group_conditions(part_b_conn, "part_b", "统一仓库", "统一名称", "统一日期")

rich.print(part_a_group_col_rst)

{
    'group1_col': [
        '安徽合肥仓',
        '河南新乡仓',
        '河南漯河仓',
        '安徽淮南仓',
        '甘肃兰州仓',
        '安徽宿州仓',
        '内蒙古呼和浩特仓',
        '河北邯郸仓',
        '河北廊坊仓',
        '河北石家庄仓'
    ],
    'group2_col': [
        '徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN',
        '熊博士果Q弹橡皮糖综合水果袋装@6(10x60g) N1 CN',
        'DODO综合果味棒棒糖零食店@2x3Kg CN',
        '熊博士白桃乌龙茶味果肉口嚼糖9粒条装@6(12x36g) CN',
        '徐福记咸香芝士味米果卷金点@4x1.5kg N1 CN',
        '熊博士综合果汁大棒糖零食店@12x76g CN',
        '趣满果花心杂果味迷你筒开盒有礼竖版@6(8x30g) CN',
        '徐福记沙琪玛香酥鸡蛋味八庄捆绑袋装@20(2x160g) N1 CN',
        '徐福记香烤牛排味米果卷金点@4x1.5kg N1 CN',
        '熊博士青提绿茶味果肉口嚼糖9粒条装@6(12x36g) CN',
        '徐福记卷心酥香草牛奶味袋装扫码专供@4(6x52g) CN',
        '徐福记糖霜橡皮糖蚯蚓型散糖金致系列@4x1.5kg CN',
        '徐福记卷心酥清新草莓味袋装扫码专供@4(6x52g) CN',
        '奇巧威化黑巧克力纸袋装零食店@4(8x46g) CN',
        '趣满果黑加仑味迷你筒开盒有礼竖版@6(8x30g) CN',
        '雀巢麦丽素可可味桶装零食店@24x108g CN',
        '奇巧威化牛奶巧克力纸袋装零食店@4(8x46g) CN',
        '徐福记黑芝麻味酥心糖散糖金致系列@4x1.5kg CN',
        '雀巢奇欧比原味零糖代可可脂巧克力散装@2x3kg CN',
        '徐福记落花生味酥心糖散糖金致系列@4x1.5kg CN',
        '奇欧比芝麻酱馅八角系列代可可脂巧克力制品散装@2x3kg CN',
        '徐福记香烤牛排味米果卷散点银致系列@4x1.5kg N1 CN',
        '徐福记咸香芝士味米果卷散点银致系列@4x1.5kg N1 CN',
        '趣满果黑加仑味迷你筒开盒有礼@6(8x30g) CN',
        '趣满果花心杂果味迷你筒开盒有礼@6(8x30g) CN',
        '徐福记橡皮糖大桔瓣型袋装零食店@20x80g CN',
        '徐福记沙琪玛松软鸡蛋味袋装@5(20x32g) N1 CN',
        '雀巢奇欧比可可味麦丽素代可可脂巧克力制品袋装@8(10x35g) CN'
    ],
    'group3_col': [
        Timestamp('2023-10-01 00:00:00'),
        Timestamp('2023-11-01 00:00:00'),
        Timestamp('2023-12-01 00:00:00'),
        Timestamp('2024-01-01 00:00:00'),
        Timestamp('2024-02-01 00:00:00'),
        Timestamp('2024-03-01 00:00:00'),
        Timestamp('2024-04-01 00:00:00'),
        Timestamp('2024-05-01 00:00:00'),
        Timestamp('2024-06-01 00:00:00'),
        Timestamp('2024-07-01 00:00:00'),
        Timestamp('2024-08-01 00:00:00'),
        Timestamp('2024-09-01 00:00:00'),
        Timestamp('2024-10-01 00:00:00'),
        Timestamp('2024-11-01 00:00:00'),
        Timestamp('2024-12-01 00:00:00')
    ]
}

### 分组提取匹配情况

In [3]:
import rich
from source.reconcile_lists import reconcile_lists
from source.get_group_data import get_group_data

group1_exit = False
group2_exit = False

for group1 in part_a_group_col_rst["group1_col"]:
    if group1_exit:
        break
    group1_exit = True
    for group2 in part_a_group_col_rst["group2_col"]:
        if group2_exit:
            break
        group2_exit = True
        for group3 in part_a_group_col_rst["group3_col"]:
            list_a = get_group_data(part_a_conn, "part_a", "统一仓库", group1, "统一名称", group2, "统一日期", group3, "收货数量")
            list_b = get_group_data(part_b_conn, "part_b", "统一仓库", group1, "统一名称", group2, "统一日期", group3, "实发数量")
            rst = reconcile_lists(list_a, list_b)
            if all(len(value) == 0 for value in rst.values()):
                continue
            rich.print(f"{group1}=>{group2}=>{group3}", rst)

安徽合肥仓=>徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN=>2023-11-01 00:00:00
{
    'exact_matches': {300.0: 1, 150.0: 1},
    'combo_matches_a_to_b': [],
    'combo_matches_b_to_a': [],
    'unmatched_a': [],
    'unmatched_b': [358.0, 300.0, 292.0, 200.0, 150.0, 150.0, 132.0, 92.0, 68.0, 8.0]
}

安徽合肥仓=>徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN=>2023-12-01 00:00:00
{
    'exact_matches': {170.0: 1, 100.0: 1, 110.0: 1},
    'combo_matches_a_to_b': [],
    'combo_matches_b_to_a': [],
    'unmatched_a': [],
    'unmatched_b': [150.0, 150.0, 100.0, 60.0, 50.0]
}

安徽合肥仓=>徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN=>2024-01-01 00:00:00
{
    'exact_matches': {109.0: 1},
    'combo_matches_a_to_b': [],
    'combo_matches_b_to_a': [],
    'unmatched_a': [611.0, 100.0],
    'unmatched_b': [720.0, 710.0, 500.0, 400.0, 380.0, 120.0, 50.0, 40.0]
}

安徽合肥仓=>徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN=>2024-03-01 00:00:00
{
    'exact_matches': {},
    'combo_matches_a_to_b': [],
    'combo_matches_b_to_a': [],
    'unmatched_a': [],
    'unmatched_b': [100.0, 54.0]
}

安徽合肥仓=>徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN=>2024-04-01 00:00:00
{
    'exact_matches': {80.0: 1, 100.0: 1, 70.0: 2},
    'combo_matches_a_to_b': [],
    'combo_matches_b_to_a': [],
    'unmatched_a': [80.0],
    'unmatched_b': [
        150.0,
        104.0,
        100.0,
        100.0,
        100.0,
        90.0,
        61.0,
        60.0,
        50.0,
        46.0,
        37.0,
        30.0,
        29.0,
        20.0,
        20.0,
        16.0,
        8.0,
        4.0
    ]
}

...

安徽合肥仓=>徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN=>2024-08-01 00:00:00
{
    'exact_matches': {},
    'combo_matches_a_to_b': [
        {'b': 120.0, 'a': [28.0, 28.0, 28.0, 28.0, 8.0]},
        {'b': 80.0, 'a': [28.0, 28.0, 24.0]}
    ],
    'combo_matches_b_to_a': [],
    'unmatched_a': [28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 24.0, 8.0],
    'unmatched_b': [120.0, 94.0, 80.0]
}

安徽合肥仓=>徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN=>2024-09-01 00:00:00
{
    'exact_matches': {4.0: 1},
    'combo_matches_a_to_b': [
        {'b': 200.0, 'a': [35.0, 35.0, 28.0, 28.0, 28.0, 28.0, 18.0]},
        {'b': 100.0, 'a': [28.0, 28.0, 28.0, 16.0]},
        {'b': 88.0, 'a': [28.0, 28.0, 24.0, 8.0]},
        {'b': 80.0, 'a': [28.0, 26.0, 16.0, 10.0]},
        {'b': 72.0, 'a': [28.0, 28.0, 16.0]},
        {'b': 50.0, 'a': [28.0, 22.0]}
    ],
    'combo_matches_b_to_a': [],
    'unmatched_a': [28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 28.0],
    'unmatched_b': [70.0, 50.0, 41.0, 36.0]
}

安徽合肥仓=>徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN=>2024-10-01 00:00:00
{
    'exact_matches': {},
    'combo_matches_a_to_b': [
        {'b': 100.0, 'a': [28.0, 28.0, 28.0, 16.0]},
        {'b': 90.0, 'a': [28.0, 28.0, 28.0, 6.0]}
    ],
    'combo_matches_b_to_a': [],
    'unmatched_a': [28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 16.0, 16.0],
    'unmatched_b': [100.0, 100.0, 50.0]
}

安徽合肥仓=>徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN=>2024-11-01 00:00:00
{
    'exact_matches': {28.0: 1},
    'combo_matches_a_to_b': [
        {'b': 100.0, 'a': [28.0, 28.0, 28.0, 16.0]},
        {'b': 82.0, 'a': [28.0, 28.0, 26.0]},
        {'b': 57.0, 'a': [28.0, 28.0, 1.0]},
        {'b': 55.0, 'a': [28.0, 27.0]},
        {'b': 50.0, 'a': [28.0, 22.0]},
        {'b': 33.0, 'a': [28.0, 5.0]},
        {'b': 30.0, 'a': [17.0, 13.0]}
    ],
    'combo_matches_b_to_a': [],
    'unmatched_a': [28.0, 28.0, 24.0, 22.0],
    'unmatched_b': [25.0, 9.0]
}

安徽合肥仓=>徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN=>2024-12-01 00:00:00
{
    'exact_matches': {1.0: 1, 20.0: 2, 23.0: 1, 25.0: 1, 28.0: 1},
    'combo_matches_a_to_b': [
        {'b': 100.0, 'a': [28.0, 28.0, 28.0, 16.0]},
        {'b': 80.0, 'a': [28.0, 28.0, 24.0]},
        {'b': 76.0, 'a': [28.0, 28.0, 16.0, 4.0]},
        {'b': 52.0, 'a': [28.0, 24.0]},
        {'b': 50.0, 'a': [26.0, 16.0, 8.0]},
        {'b': 35.0, 'a': [28.0, 5.0, 2.0]},
        {'b': 30.0, 'a': [15.0, 15.0]}
    ],
    'combo_matches_b_to_a': [],
    'unmatched_a': [28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 16.0, 12.0, 12.0, 12.0, 12.0, 8.0, 4.0, 1.0],
    'unmatched_b': [100.0, 100.0, 100.0]
}

### 单点测试

In [6]:
import rich
from source.reconcile_lists import reconcile_lists
from source.get_group_data import get_group_data
from collections import Counter

list_a = get_group_data(part_a_conn, "part_a", "统一仓库", "安徽合肥仓", "统一名称", "徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN", "统一日期", "2024-07-01 00:00:00", "收货数量")
list_b = get_group_data(part_b_conn, "part_b", "统一仓库", "安徽合肥仓", "统一名称", "徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN", "统一日期", "2024-07-01 00:00:00", "实发数量")
_rst = reconcile_lists(list_a, list_b)
rich.print(_rst)

{
    'exact_matches': {10.0: 1},
    'combo_matches_a_to_b': [
        {'b': 120.0, 'a': [32.0, 29.0, 28.0, 23.0, 8.0]},
        {'b': 70.0, 'a': [28.0, 28.0, 14.0]},
        {'b': 35.0, 'a': [28.0, 7.0]}
    ],
    'combo_matches_b_to_a': [],
    'unmatched_a': [28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 28.0, 10.0],
    'unmatched_b': [107.0, 61.0, 20.0]
}

In [ ]:
print(Counter(_rst["exact_matches"]))
print(Counter(_rst["exact_matches"]))
print(Counter(_rst["unmatched_a"]))
print(Counter(_rst["unmatched_b"]))